# Spearman correlation

In this notebook, we compute the Spearman correlations between the time-series of the administrative regions of Nigeria. Through this analysis, we can identify time-series with high correlations between them and then decide to exclude some for subsequent analyzes.

In [1]:
import dataframe_image as dfi
import pandas as pd
import numpy as np

In [2]:
COUNTRY = "Nigeria"

In [3]:
PATH_TO_DATA_FOLDER = "../Dataset time-series/output_data/" + COUNTRY + "/"

## Time-series dataset

In [4]:
# Load the time-series data (daily interpolation).
df = pd.read_csv(PATH_TO_DATA_FOLDER + COUNTRY + "-day.csv", header = [0, 1], index_col = 0)
df.index.name = "Datetime"
df.index = pd.to_datetime(df.index)
freq = "D"
df.index.freq = freq

In [5]:
df.columns.get_level_values(1).unique()

Index(['1 Month Anomaly Rainfalls (%)', '3 Months Anomaly Rainfalls (%)',
       'Code', 'FCG <= 2', 'Fatalities', 'Lat', 'Lon', 'NDVI', 'NDVI Anomaly',
       'Population', 'Price cereals and tubers', 'Rainfalls (mm)', 'Ramadan',
       'rCSI >= 19'],
      dtype='object', name='Indicator')

In [6]:
# Consider only some indicators for the analysis with Spearman correlation.
INDICATORS_TO_CONSIDER = ["FCG <= 2", "1 Month Anomaly Rainfalls (%)", "3 Months Anomaly Rainfalls (%)", "NDVI", "Price cereals and tubers", "Fatalities", "NDVI Anomaly", "rCSI >= 19", "Rainfalls (mm)"]
df = pd.concat([df.loc[:, (slice(None), feature)] for feature in INDICATORS_TO_CONSIDER], axis = 1).sort_index(axis = 1)
df

AdminStrata                       Adamawa                                 \
Indicator   1 Month Anomaly Rainfalls (%) 3 Months Anomaly Rainfalls (%)   
Datetime                                                                   
2019-01-18                     100.259677                      93.037140   
2019-01-19                     100.244839                      93.977969   
2019-01-20                     100.230000                      95.031538   
2019-01-21                     100.215191                      96.061030   
2019-01-22                     100.200411                      97.066445   
...                                   ...                            ...   
2020-09-26                      77.297220                      89.440721   
2020-09-27                      78.097507                      89.420572   
2020-09-28                      78.983166                      89.407210   
2020-09-29                      79.954197                      89.400633   
2020-09-30                      81.010602                      89.400844   

AdminStrata                                               \
Indicator     FCG <= 2 Fatalities      NDVI NDVI Anomaly   
Datetime                                                   
2019-01-18   23.586938        0.0  0.334429    96.868029   
2019-01-19   22.755425        0.0  0.332291    96.927069   
2019-01-20   22.547817        0.0  0.330208    96.985875   
2019-01-21   23.758271        0.0  0.328135    97.044913   
2019-01-22   22.374279        0.0  0.326072    97.104183   
...                ...        ...       ...          ...   
2020-09-26   41.470788        0.0  0.649718    96.235517   
2020-09-27   45.390480        0.0  0.650146    96.312743   
2020-09-28   45.110420        0.0  0.650531    96.387221   
2020-09-29   45.623021        0.0  0.650872    96.458949   
2020-09-30   47.784233        0.0  0.651171    96.527928   

AdminStrata                                                     \
Indicator   Price cereals and tubers Rainfalls (mm) rCSI >= 19   
Datetime                                                         
2019-01-18                  0.204482         0.0000  43.008741   
2019-01-19                  0.204677         0.0000  43.619385   
2019-01-20                  0.204873         0.0000  42.481559   
2019-01-21                  0.205068         0.0006  42.558962   
2019-01-22                  0.205264         0.0012  41.129399   
...                              ...            ...        ...   
2020-09-26                  0.842524       206.0412  16.702326   
2020-09-27                  0.841555       204.7794  16.707763   
2020-09-28                  0.840585       203.5176  19.095040   
2020-09-29                  0.839615       202.2558  20.885011   
2020-09-30                  0.838645       200.9940  21.148869   

AdminStrata                         Borno  ...             \
Indicator   1 Month Anomaly Rainfalls (%)  ... rCSI >= 19   
Datetime                                   ...              
2019-01-18                     100.000000  ...  35.259528   
2019-01-19                     100.000000  ...  34.719999   
2019-01-20                     100.000000  ...  34.653470   
2019-01-21                     100.000000  ...  34.362899   
2019-01-22                     100.000000  ...  36.355709   
...                                   ...  ...        ...   
2020-09-26                     114.770357  ...  11.992484   
2020-09-27                     115.029275  ...  13.156250   
2020-09-28                     115.303064  ...  14.377357   
2020-09-29                     115.591724  ...  14.936559   
2020-09-30                     115.895256  ...  14.129691   

AdminStrata                          Yobe                                 \
Indicator   1 Month Anomaly Rainfalls (%) 3 Months Anomaly Rainfalls (%)   
Datetime                                                                   
2019-01-18                     100.000000                     143.249296   
2019-01-19

## Correlations

### Nature indicators

In [7]:
select = df.columns.get_level_values(1).isin(["NDVI", "NDVI Anomaly", "Rainfall (mm)", "1 Month Anomaly Rainfalls (%)", "3 Months Anomaly Rainfalls (%)"])
df_nature = df.loc[:, select]
df_nature.head()

AdminStrata                       Adamawa                                 \
Indicator   1 Month Anomaly Rainfalls (%) 3 Months Anomaly Rainfalls (%)   
Datetime                                                                   
2019-01-18                     100.259677                      93.037140   
2019-01-19                     100.244839                      93.977969   
2019-01-20                     100.230000                      95.031538   
2019-01-21                     100.215191                      96.061030   
2019-01-22                     100.200411                      97.066445   

AdminStrata                                                Borno  \
Indicator        NDVI NDVI Anomaly 1 Month Anomaly Rainfalls (%)   
Datetime                                                           
2019-01-18   0.334429    96.868029                         100.0   
2019-01-19   0.332291    96.927069                         100.0   
2019-01-20   0.330208    96.985875                         100.0   
2019-01-21   0.328135    97.044913                         100.0   
2019-01-22   0.326072    97.104183                         100.0   

AdminStrata                                                        \
Indicator   3 Months Anomaly Rainfalls (%)      NDVI NDVI Anomaly   
Datetime                                                            
2019-01-18                      117.367238  0.267711   102.864071   
2019-01-19                      117.995892  0.266671   102.866521   
2019-01-20                      118.633264  0.265653   102.869594   
2019-01-21                      119.148946  0.264630   102.874483   
2019-01-22                      119.542937  0.263603   102.881188   

AdminStrata                          Yobe                                 \
Indicator   1 Month Anomaly Rainfalls (%) 3 Months Anomaly Rainfalls (%)   
Datetime                                                                   
2019-01-18                          100.0                     143.249296   
2019-01-19                          100.0                     143.210813   
2019-01-20                          100.0                     143.053244   
2019-01-21                          100.0                     142.693166   
2019-01-22                          100.0                     142.130581   

AdminStrata                         
Indicator        NDVI NDVI Anomaly  
Datetime                            
2019-01-18   0.232465    99.634913  
2019-01-19   0.231498    99.533540  
2019-01-20   0.230541    99.430884  
2019-01-21   0.229593    99.333907  
2019-01-22   0.228655    99.242609

In [8]:
# Compute the correlation between the nature time-series for the various administrative regions: we then obtain a list of correlation matrices equal to the number of administrative regions.
corr_matrices_nature = list()

def correlation_matrices_nature(group):
    group.columns = group.columns.droplevel()
    mtrx = group.corr(method = "spearman")#.values
    corr_matrices_nature.append(mtrx)

df_nature.groupby(level = 0, axis = 1).apply(correlation_matrices_nature);

In [9]:
# We take the mean of the various correlation matrices (administrative regions).
CORR_nature_mean = pd.DataFrame(np.mean(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_nature.columns.droplevel().unique(), columns = df_nature.columns.droplevel().unique())
corr = CORR_nature_mean.style.background_gradient(cmap = "coolwarm")
corr.export_png("./output_images/nature_indicators_%s.png" % COUNTRY)
corr

Indicator,1 Month Anomaly Rainfalls (%),3 Months Anomaly Rainfalls (%),NDVI,NDVI Anomaly
Indicator,,,,
1 Month Anomaly Rainfalls (%),1.000000,0.595842,0.345953,0.076218
3 Months Anomaly Rainfalls (%),0.595842,1.000000,0.318319,0.167348
NDVI,0.345953,0.318319,1.000000,-0.077880
NDVI Anomaly,0.076218,0.167348,-0.077880,1.000000


In [10]:
# We take the median of the various correlation matrices (administrative regions).
CORR_nature_median = pd.DataFrame(np.median(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_nature.columns.droplevel().unique(), columns = df_nature.columns.droplevel().unique())
CORR_nature_median.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly Rainfalls (%),3 Months Anomaly Rainfalls (%),NDVI,NDVI Anomaly
Indicator,,,,
1 Month Anomaly Rainfalls (%),1.000000,0.604473,0.388410,0.124334
3 Months Anomaly Rainfalls (%),0.604473,1.000000,0.446368,0.338059
NDVI,0.388410,0.446368,1.000000,-0.120673
NDVI Anomaly,0.124334,0.338059,-0.120673,1.000000


In [11]:
# We take the variance of the various correlation matrices (administrative regions).
CORR_nature_variance = pd.DataFrame(np.var(np.stack(corr_matrices_nature, axis = 0), axis = 0), index = df_nature.columns.droplevel().unique(), columns = df_nature.columns.droplevel().unique())
CORR_nature_variance.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly Rainfalls (%),3 Months Anomaly Rainfalls (%),NDVI,NDVI Anomaly
Indicator,,,,
1 Month Anomaly Rainfalls (%),0.000000,0.000193,0.024009,0.058005
3 Months Anomaly Rainfalls (%),0.000193,0.000000,0.055066,0.067709
NDVI,0.024009,0.055066,0.000000,0.128062
NDVI Anomaly,0.058005,0.067709,0.128062,0.000000


### All indicators

In [12]:
# Compute the correlation between all the time-series for the various administrative regions: we then obtain a list of correlation matrices equal to the number of administrative regions.
corr_matrices = list()

def correlation_matrices(group):
    group.columns = group.columns.droplevel()
    mtrx = group.corr(method = "spearman").values
    corr_matrices.append(mtrx)

df.groupby(level = 0, axis = 1).apply(correlation_matrices);

In [13]:
# We take the mean of the various correlation matrices (administrative regions).
CORR_mean = pd.DataFrame(np.mean(np.stack(corr_matrices, axis = 0), axis = 0), index = df.columns.droplevel().unique(), columns = df.columns.droplevel().unique())
corr = CORR_mean.style.background_gradient(cmap = "coolwarm")
corr.export_png("./output_images/all_indicators_%s.png" % COUNTRY)
corr

Indicator,1 Month Anomaly Rainfalls (%),3 Months Anomaly Rainfalls (%),FCG <= 2,Fatalities,NDVI,NDVI Anomaly,Price cereals and tubers,Rainfalls (mm),rCSI >= 19
Indicator,,,,,,,,,
1 Month Anomaly Rainfalls (%),1.000000,0.595842,-0.220405,-0.258842,0.345953,0.076218,-0.149659,0.358118,0.223438
3 Months Anomaly Rainfalls (%),0.595842,1.000000,-0.306089,-0.052421,0.318319,0.167348,-0.267680,-0.013940,0.101630
FCG <= 2,-0.220405,-0.306089,1.000000,0.326950,-0.152326,-0.048154,0.409736,0.085097,-0.214853
Fatalities,-0.258842,-0.052421,0.326950,1.000000,-0.180462,0.294727,0.165891,-0.216410,-0.128283
NDVI,0.345953,0.318319,-0.152326,-0.180462,1.000000,-0.077880,0.238114,0.541229,-0.179323
NDVI Anomaly,0.076218,0.167348,-0.048154,0.294727,-0.077880,1.000000,0.052842,-0.419658,-0.321005
Price cereals and tubers,-0.149659,-0.267680,0.409736,0.165891,0.238114,0.052842,1.000000,0.230016,-0.636220
Rainfalls (mm),0.358118,-0.013940,0.085097,-0.216410,0.541229,-0.419658,0.230016,1.000000,0.109870
rCSI >= 19,0.223438,0.101630,-0.214853,-0.128283,-0.179323,-0.321005,-0.636220,0.109870,1.000000


In [14]:
# We take the median of the various correlation matrices (administrative regions).
CORR_median = pd.DataFrame(np.median(np.stack(corr_matrices, axis = 0), axis = 0), index = df.columns.droplevel().unique(), columns = df.columns.droplevel().unique())
CORR_median.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly Rainfalls (%),3 Months Anomaly Rainfalls (%),FCG <= 2,Fatalities,NDVI,NDVI Anomaly,Price cereals and tubers,Rainfalls (mm),rCSI >= 19
Indicator,,,,,,,,,
1 Month Anomaly Rainfalls (%),1.000000,0.604473,-0.333627,-0.234840,0.388410,0.124334,-0.155359,0.438908,0.208437
3 Months Anomaly Rainfalls (%),0.604473,1.000000,-0.410258,-0.084587,0.446368,0.338059,-0.354061,-0.001156,0.069267
FCG <= 2,-0.333627,-0.410258,1.000000,0.188191,-0.136421,-0.010587,0.408530,0.078962,-0.091154
Fatalities,-0.234840,-0.084587,0.188191,1.000000,-0.125883,0.275208,0.169111,-0.353719,-0.212290
NDVI,0.388410,0.446368,-0.136421,-0.125883,1.000000,-0.120673,0.179893,0.549443,-0.284396
NDVI Anomaly,0.124334,0.338059,-0.010587,0.275208,-0.120673,1.000000,0.279831,-0.581841,-0.321817
Price cereals and tubers,-0.155359,-0.354061,0.408530,0.169111,0.179893,0.279831,1.000000,0.203268,-0.654856
Rainfalls (mm),0.438908,-0.001156,0.078962,-0.353719,0.549443,-0.581841,0.203268,1.000000,0.122794
rCSI >= 19,0.208437,0.069267,-0.091154,-0.212290,-0.284396,-0.321817,-0.654856,0.122794,1.000000


In [15]:
# We take the variance of the various correlation matrices (administrative regions).
CORR_variance = pd.DataFrame(np.var(np.stack(corr_matrices, axis = 0), axis = 0), index = df.columns.droplevel().unique(), columns = df.columns.droplevel().unique())
CORR_variance.style.background_gradient(cmap = "coolwarm")

Indicator,1 Month Anomaly Rainfalls (%),3 Months Anomaly Rainfalls (%),FCG <= 2,Fatalities,NDVI,NDVI Anomaly,Price cereals and tubers,Rainfalls (mm),rCSI >= 19
Indicator,,,,,,,,,
1 Month Anomaly Rainfalls (%),0.000000,0.000193,0.043082,0.003886,0.024009,0.058005,0.002586,0.027005,0.018034
3 Months Anomaly Rainfalls (%),0.000193,0.000000,0.025592,0.004702,0.055066,0.067709,0.019684,0.042498,0.068179
FCG <= 2,0.043082,0.025592,0.000000,0.046789,0.003619,0.072720,0.027065,0.000775,0.033134
Fatalities,0.003886,0.004702,0.046789,0.000000,0.077480,0.015280,0.116134,0.053620,0.061177
NDVI,0.024009,0.055066,0.003619,0.077480,0.000000,0.128062,0.008495,0.013928,0.030538
NDVI Anomaly,0.058005,0.067709,0.072720,0.015280,0.128062,0.000000,0.124801,0.135119,0.067746
Price cereals and tubers,0.002586,0.019684,0.027065,0.116134,0.008495,0.124801,0.000000,0.005962,0.002674
Rainfalls (mm),0.027005,0.042498,0.000775,0.053620,0.013928,0.135119,0.005962,0.000000,0.003649
rCSI >= 19,0.018034,0.068179,0.033134,0.061177,0.030538,0.067746,0.002674,0.003649,0.000000
